In [41]:
# Import initial dependencies
import pandas as pd
import numpy as np

In [42]:
# Convert CSVs to DFs
feature_csv = "feature_df.csv"
target_csv = "target_df.csv"

feature_csv_df = pd.read_csv(feature_csv)
target_csv_df = pd.read_csv(target_csv)

In [43]:
# Check feature DF head
feature_csv_df.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_steff_err1,koi_model_snr
0,0,0,0,81,25.8
1,0,1,0,158,76.3
2,0,1,0,157,505.6
3,0,0,0,169,40.9
4,0,0,0,189,40.2


In [44]:
# Check feature DF shape
feature_csv_df.shape

(6991, 5)

In [45]:
# Check target DF head
target_csv_df.head()

,koi_disposition
0,CONFIRMED
1,FALSE POSITIVE
2,FALSE POSITIVE
3,CONFIRMED
4,CONFIRMED


In [46]:
# Check target DF shape
target_csv_df.shape

(6991, 1)

# SPLIT, NORMALIZE, AND ENCODE THE DATA

In [47]:
# Assign the features and target to X and y abd check assignments
raw_feature_data = feature_csv_df.values
raw_target_data = target_csv_df.values
X = raw_feature_data[:, 0:6]
y = raw_target_data[:, 0]

print(X, y)

[[  0.    0.    0.   81.   25.8]
 [  0.    1.    0.  158.   76.3]
 [  0.    1.    0.  157.  505.6]
 ...
 [  0.    0.    0.  165.   10.6]
 [  0.    0.    1.  193.   12.3]
 [  0.    0.    1.  158.    8.2]] ['CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE' ... 'CANDIDATE'
 'FALSE POSITIVE' 'FALSE POSITIVE']


In [48]:
# Create the train and test sets for the features and target
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [49]:
# Scale the features using the MinMax scaler since we know their values
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [50]:
# Encode the target for the train and test set so it can be fed into our NN and conver to a vector and check it
label_encoder = LabelEncoder()
label_encoder.fit(y_train)


encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

encoded_y_train

array([0, 2, 2, ..., 2, 2, 2])

In [51]:
# Convert encoded targets to one-hot-encoding and check it
from keras.utils import to_categorical

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

y_train_categorical

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

# HYPERPERAMETER TUNING

In [52]:
# Import the sequential and dense modules fo build my NN
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [53]:
# Create model and print params so I can see what can be tuned
knn = KNeighborsClassifier()

knn.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [54]:
# Create param grid
param_grid = {'n_neighbors': [x for x in range(1, 102, 2)],
               'metric': ['minkowski','euclidean','manhattan'],
               'weights': ['uniform','distance']}

# Assign the optimizer grid search parameters, fit, and run the model
grid = GridSearchCV(estimator=knn, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train_categorical)

C:\Users\howar\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [55]:
# Print the score and best params for the KNN model
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.840931 using {'metric': 'minkowski', 'n_neighbors': 21, 'weights': 'uniform'}


In [56]:
# Print the score for the traning vs. test data
print(f"Training Data Score: {grid_result.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {grid_result.score(X_test_scaled, y_test_categorical)}")

Training Data Score: 0.8502765592218196
Testing Data Score: 0.8323798627002288


# SAVE THE MODEL

In [57]:
# Save the tuned model for future use
import joblib
filename = 'Howard_Mitchell_KNN.sav'
joblib.dump(grid_result, filename)

['Howard_Mitchell_KNN.sav']